# save the images as video
# Basler ace camera

In [2]:
# import packages and functions
from pypylon import pylon
from imageio import get_writer
import numpy as np
import os
import cv2
from datetime import datetime
import time
import matplotlib.pyplot as plt
import sys
from camcommands import camOpen
from multiprocessing import Process
from threading import Thread

connect camera with OpenCV

In [14]:
# list cameras connected with PC
# 0. set number of cameras to use
cameras_2_use = 2

# 1. get transport layer and all attached devices
tl_factory = pylon.TlFactory.GetInstance()
devices = tl_factory.EnumerateDevices() #enumerate devices connected

# 1.1 soft ware issue from pylon. this block may be deleted later if the issue is solved
# here, get instance and enumerate devices until two different cameras are recognized
temp_dev_list = []
if len(devices) == 0:
    raise pylon.RuntimeException("No camera connected, please check the camera connection!")
else:
    for i, dev_info in enumerate(devices):
        temp_dev_list.append(dev_info.GetSerialNumber())
    while len(set(temp_dev_list)) != 2: # number of cameras == 2
        tl_factory = pylon.TlFactory.GetInstance()
        devices = tl_factory.EnumerateDevices()
        temp_dev_list = []
        for i, dev_info in enumerate(devices):
            temp_dev_list.append(dev_info.GetSerialNumber())
        print(temp_dev_list)
        if len(set(temp_dev_list)) == 2:
            break

print(temp_dev_list)


"""
# 1.1 software issue from pylon. this block may be deleted later
# remove duplicated camera list
set_devices = []
for device in devices:
    if device not in set_devices:
        set_devices.append(device)
    else:
        pass
set_devices = tuple(set_devices)
"""

# 2. check the devices information
if len(devices) == 0:
    raise pylon.RuntimeException("No camera connected")
else:
    print("total number of devices:", len(devices)) # print total number of devices connected
    for i, dev_info in enumerate(devices): # 
        if dev_info.GetDeviceClass() == 'BaslerGigE': # check if the connected cameras are GigE model. Otherwise software trigger won't work
            print("using %s as IP %s SN %s" % (dev_info.GetModelName(), dev_info.GetIpAddress() ,dev_info.GetSerialNumber()))
        else:
            raise EnvironmentError("no GigE device found")

# 3. create instant array and attach all devices on the Array
cams = pylon.InstantCameraArray(min(len(devices), cameras_2_use)) # create instant
for i, cam in enumerate(cams):
    cam.Attach(tl_factory.CreateDevice(devices[i]))

['21566554', '21585923']
['21566554', '21585923']
total number of devices: 2
using acA1300-30gc as IP 169.254.91.231 SN 21566554
using acA1300-30gc as IP 169.254.4.51 SN 21585923


In [15]:
# setting camera's parameters
fps = 10 # frame per sec
ExposureTime = 5000 # in µs
Height, width = 962, 1286 # in pixel
pixelformat = "Mono8" # black and white image

for cam in cams:
    cam.Open() # open camera to change the parameter
    print("Setting device ", cam.GetDeviceInfo().GetFriendlyName())
    print("original: ", "Height:",cam.Height.GetValue(), "Width:", cam.Width.GetValue(), 
            "Exposuretime:", cam.ExposureTimeRaw.GetValue(), "AcquisitionFrameRate:", cam.AcquisitionFrameRateAbs.GetValue(), "pixelformat:", cam.PixelFormat.GetValue())
    cam.Height.SetValue(Height)
    cam.Width.SetValue(width)
    cam.ExposureTimeRaw.SetValue(ExposureTime)
    cam.AcquisitionFrameRateAbs.SetValue(fps)
    cam.PixelFormat.SetValue(pixelformat)
    print("Set value: ", "Height:",cam.Height.GetValue(), "Width:", cam.Width.GetValue(), 
            "Exposuretime:", cam.ExposureTimeRaw.GetValue(), "AcquisitionFrameRate:", cam.AcquisitionFrameRateAbs.GetValue())
    cam.Close() # close camera
    

Setting device  acA1300_SN01 (21566554)
original:  Height: 962 Width: 1286 Exposuretime: 5000 AcquisitionFrameRate: 20.0
Set value:  Height: 962 Width: 1286 Exposuretime: 5000 AcquisitionFrameRate: 10.0
Setting device  acA1300_SN03 (21585923)
original:  Height: 962 Width: 1286 Exposuretime: 5000 AcquisitionFrameRate: 20.0
Set value:  Height: 962 Width: 1286 Exposuretime: 5000 AcquisitionFrameRate: 10.0


In [18]:
# capturing video with multiple cameras simultaneously
video_pos = {0:"front", 1:"side"}

#function for simple video recording
# fps = 20 # fps is set already
video_format = "FFMPEG"
video_container = "mp4"
#video_codec = "h264"
#writing_mode = "I" # imageio writing mode "I" for video recording
#macro_block_size = 2 # integer, width and height should be divisable with this number
quality = 10  # float 0 - 10, default 5. better resolution with higher number
bitrate = None  # integer, if None, quality parameter will be used. Otherwise, quality parameter will be ignored


def video_recording(file_dir, filename, caminstance):
    #caminstance.StopGrabbing()
    with get_writer(os.path.join(file_dir, filename), format=video_format, quality=quality, bitrate=bitrate) as writer:
        #fps=fps, codec=video_codec, macro_block_size=macro_block_size, mode=writing_mode,) as writer:
        print("parent process : %s / process id %s" % (os.getppid(), os.getpid()))
        print("recording start with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(50)
        while caminstance.IsGrabbing():
            try :
                res = caminstance.RetrieveResult(10000, pylon.TimeoutHandling_ThrowException)
                print(res.Array)
            except:
                print("something wrong while recording")
            writer.append_data(res.Array)
            res.Release()
        print("recording finish with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.Close()

cam1 = Thread(name="cam1", target=video_recording, 
            args=("C:/Users/dkim/Desktop/basler_cam/recording", "test_multicore_%s.mp4" % (video_pos[0]), cams[0]))
cam2 = Thread(name="cam2", target=video_recording, 
                args=("C:/Users/dkim/Desktop/basler_cam/recording", "test_multicore_%s.mp4" % (video_pos[1]), cams[1]))

cam1.start()
cam2.start()

#cam1.join()
#cam2.join()

#while True:
#    time.sleep(2)

parent process : 684 / process id 10860
parent process : 684 / process id 10860
recording start with acA1300_SN01 (21566554) at 2021-11-10 17:03:25.018588
recording start with acA1300_SN03 (21585923) at 2021-11-10 17:03:25.018588


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


[[64 63 65 ...  7  7  6]
 [63 62 64 ...  6  6  7]
 [64 62 64 ...  7  7  7]
 ...
 [ 3  3  3 ...  4  4  4]
 [ 2  3  3 ...  5  4  5]
 [ 3  2  3 ...  5  4  5]]
[[49 50 51 ...  1  1  2]
 [50 49 50 ...  1  1  1]
 [49 50 50 ...  2  2  1]
 ...
 [ 7  7  8 ... 86 83 84]
 [ 8  7  8 ... 86 84 87]
 [ 8  8  8 ... 88 88 88]]
[[63 62 63 ... 30 30 32]
 [62 61 61 ... 29 29 30]
 [62 62 62 ... 29 29 29]
 ...
 [ 2  2  2 ...  4  4  5]
 [ 2  2  2 ...  5  4  4]
 [ 2  2  2 ...  6  6  5]]
[[47 48 48 ...  2  1  1]
 [49 49 48 ...  2  1  1]
 [50 50 50 ...  1  1  1]
 ...
 [ 8  8  8 ... 69 70 72]
 [ 8  8  7 ... 70 71 73]
 [ 8  8  7 ... 73 74 73]]
[[46 47 46 ...  1  2  1]
 [46 46 46 ...  2  1  1]
 [45 47 46 ...  2  1  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ... 44 45 48]
 [ 9  8  8 ... 47 47 47]]
[[61 62 62 ... 11 10  9]
 [61 62 61 ... 12 10  9]
 [61 61 60 ... 12 11 10]
 ...
 [ 2  1  1 ... 41 40 41]
 [ 2  1  1 ... 40 39 40]
 [ 2  2  2 ... 38 40 42]]
[[51 51 52 ...  1  1  1]
 [51 51 50 ...  1  1  2]
 [51 51 50 ...

Exception in thread cam1:
Traceback (most recent call last):
  File "C:\Users\dkim\Anaconda3\envs\imagetrack\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\dkim\Anaconda3\envs\imagetrack\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-0d8fc35be3e3>", line 30, in video_recording
  File "C:\Users\dkim\Anaconda3\envs\imagetrack\lib\site-packages\imageio\core\format.py", line 502, in append_data
    return self._append_data(im, total_meta)
  File "C:\Users\dkim\Anaconda3\envs\imagetrack\lib\site-packages\imageio\plugins\ffmpeg.py", line 565, in _append_data
    raise ValueError("All images in a movie should have same size")
ValueError: All images in a movie should have same size


[[63 65 64 ...  2  1  1]
 [62 66 64 ...  1  1  1]
 [61 64 63 ...  1  1  2]
 ...
 [ 8  8  9 ... 10 10  9]
 [ 8  8  8 ... 10 10 11]
 [ 8  7  7 ...  9 10 11]]
[]
[[51 52 52 ... 34 34 33]
 [52 50 51 ... 35 35 33]
 [51 50 51 ... 36 36 33]
 ...
 [ 2  2  2 ...  4  4  4]
 [ 2  2  2 ...  4  4  4]
 [ 2  2  2 ...  6  4  4]]
[[51 50 51 ... 33 32 33]
 [51 50 48 ... 33 32 32]
 [51 51 50 ... 33 32 30]
 ...
 [ 1  1  1 ...  4  3  4]
 [ 2  1  1 ...  4  4  4]
 [ 2  2  1 ...  5  4  4]]
[[48 47 47 ... 34 33 33]
 [49 48 48 ... 34 35 34]
 [48 50 49 ... 35 36 35]
 ...
 [ 1  2  2 ...  4  5  5]
 [ 2  2  2 ...  4  5  6]
 [ 2  2  2 ...  5  5  5]]
[[52 52 53 ... 28 28 29]
 [52 52 53 ... 29 28 28]
 [51 52 53 ... 29 29 30]
 ...
 [ 2  2  2 ...  4  4  4]
 [ 2  2  2 ...  5  4  4]
 [ 1  2  2 ...  4  4  4]]
[[48 47 47 ... 33 33 33]
 [47 47 46 ... 33 33 33]
 [48 47 46 ... 33 33 33]
 ...
 [ 2  2  2 ...  4  4  4]
 [ 2  2  2 ...  4  4  4]
 [ 1  1  1 ...  4  4  4]]
[[52 53 52 ... 36 37 36]
 [52 53 52 ... 36 36 35]
 [52 54 52 

In [ ]:
### this is original never change!

# capturing video with multiple cameras at simultaneously
video_pos = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    with get_writer(os.path.join(file_dir, filename)) as writer:
        print("parent process : %s / process id %s" % (os.getppid(), os.getpid()))
        print("recording start with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(50)
        print("trying camera opening")
        caminstance.Open()
        print(caminstance.Height.GetValue())
        caminstance.Close()

        print("cam opening working")


        while caminstance.IsGrabbing():
            try :
                print("working")
                res = caminstance.RetrieveResult(10000, pylon.TimeoutHandling_ThrowException)
                print(res)
            except:
                print("something wrong while recording")
            writer.append_data(res.Array)
            res.Release()
        print("recording finish with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))


cam1 = Process(name="cam1", target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[0]), cams[0]))
cam2 = Process(name="cam2", target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[1]), cams[1]))
cam1.start()
cam2.start()
#cam1.join()
#cam2.join()
#print(f'Process cam1 is alive: {cam1.is_alive()}')
#print(f'Process cam2 is alive: {cam2.is_alive()}')


parent process : 1343 / process id 2619
recording start with acA1300_SN03 (21585923) at 2021-11-04 11:28:56.480740parent process : 1343 / process id 2620

recording start with acA1300_SN01 (21566554) at 2021-11-04 11:28:56.487561
trying camera opening
962trying camera opening

cam opening working
recording finish with acA1300_SN03 (21585923) at 2021-11-04 11:28:56.614802
962
cam opening working
recording finish with acA1300_SN01 (21566554) at 2021-11-04 11:28:57.115444


In [11]:
# capturing video with multiple cameras at simultaneously
filename = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    with get_writer(os.path.join(file_dir, filename)) as writer:
        
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(100)
        while caminstance.IsGrabbing():
            res = caminstance.RetrieveResult(100)
            writer.append_data(res.Array)
            res.Release()

for i, cam in enumerate(cams):
    with get_writer("/Users/dkim/Desktop/basler_camera/recording/%s_multi_cams.avi" % (filename[i] ,) ) as writer:
        cam.StopGrabbing()
        cam.StartGrabbingMax(100)
        while cam.IsGrabbing():
            res = cam.RetrieveResult(100)
            print(res)
            writer.append_data(res.Array)
            #print(res.BlockID)
            res.Release()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c5fec60> >


[swscaler @ 0x7f8727379000] Warning: data is not aligned! This can lead to a speed loss


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8600> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon:

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c5fec60> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >


[swscaler @ 0x7f97b5179000] Warning: data is not aligned! This can lead to a speed loss


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5d5bc930> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon:

In [17]:
"""
working with multiple cameras
for the detail commands, see here: https://docs.baslerweb.com/action-commands.html#action-device-key
short paramer description
Action Device Key: 32-bit key to excute a corresponding action on camera
Action Group Key: 32-bit key for a group of devices (cameras) to excute an action
Action Group Mask: 32-bit key for a subgroup of devies

setting up the connection with multiple cameras and checking the parameters
"""

# 1. set action key, group key and group mask. As I'm using one group and all cameras in this group working simaltaneously,
# I did not set multiple action key, group key and group mask
action_key = 0x4711
group_key = 0x112233
group_mask = pylon.AllGroupMask

# 2. Initiate automatic configuration by registering ActionTriggerConfiguration.
for cam in cams:
    cam.RegisterConfiguration(
        pylon.ActionTriggerConfiguration(action_key, group_key, group_mask),
        pylon.RegistrationMode_Append,
        pylon.Cleanup_Delete
        )

# 3. Create a suitable ActionCommand object. For that a GigETransportLayer object is needed.
gige_tl = tl_factory.CreateTl('BaslerGigE')

# 4. Using default value of "255.255.255.255" for fourth parameter 'broadcastAddress'.
act_cmd = gige_tl.ActionCommand(action_key, group_key, group_mask)